In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pickle

# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

%matplotlib inline

The steps to load the required TSVs into postgres on myaws are as follows:

Rename the tab-delimited files as csv files. Then secure copy them from your local folders over to your data folder on AWS.

Locally in python, read in the first few lines of each file using pd.read_csv(). Then write the column names to a text file, so that you can then create the appropriate field names. Then copy those field names into a sql command to postgres to create the table you want.

    scp Q1_texas_2011.csv myaws:data/
    
    with open('texas_discharge_columns.txt', 'w') as f:
        for col in texas_discharge_columns:
            f.write('%s TEXT,\n' % col)

    COPY discharges_2011 FROM '/home/ubuntu/texas_discharge_2011.csv' DELIMITER E'\t' CSV;

In [15]:
%%bash
ls data/texas/

PUDF_base1_1q2011_tab.txt
PUDF_base1_2q2011_tab.txt
PUDF_base1_3q2011_tab.txt
PUDF_base1_4q2011_tab.txt
Q1_texas_2011.csv
Q2_texas_2011.csv
Q3_texas_2011.csv
Q4_texas_2011.csv


In [31]:
# Read in the column names for each csv file.
Q1_head = pd.read_csv('data/texas/Q1_texas_2011.csv', sep='\t', nrows=10)
Q2_head = pd.read_csv('data/texas/Q2_texas_2011.csv', sep='\t', nrows=10)
Q3_head = pd.read_csv('data/texas/Q3_texas_2011.csv', sep='\t', nrows=10)
Q4_head = pd.read_csv('data/texas/Q4_texas_2011.csv', sep='\t', nrows=10)

In [32]:
Q1_columns = [str(col).lower() + " TEXT" for col in Q1_head.columns]

Q1_columns_str = ", ".join(Q1_columns)
Q1_create_table_query = "CREATE TABLE q1 (" + Q1_columns_str + ")"

Q1_load_table = "COPY q1 FROM '/home/ubuntu/data/Q1_texas_2011.csv' DELIMITER E'\t' CSV"

In [32]:
Q2_columns = [str(col).lower() + " TEXT" for col in Q2_head.columns]

Q2_columns_str = ", ".join(Q2_columns)
Q2_create_table_query = "CREATE TABLE q2 (" + Q2_columns_str + ")"

Q2_load_table = "COPY q2 FROM '/home/ubuntu/data/Q2_texas_2011.csv' DELIMITER E'\t' CSV"

In [40]:
Q3_columns = [str(col).lower() + " TEXT" for col in Q3_head.columns]

Q3_columns_str = ", ".join(Q3_columns)
Q3_create_table_query = "CREATE TABLE q3 (" + Q3_columns_str + ")"

Q3_load_table = "COPY q3 FROM '/home/ubuntu/data/Q3_texas_2011.csv' DELIMITER E'\t' CSV"

In [45]:
Q4_columns = [str(col).lower() + " TEXT" for col in Q4_head.columns]

Q4_columns_str = ", ".join(Q4_columns)
Q4_create_table_query = "CREATE TABLE q4 (" + Q4_columns_str + ")"

Q4_load_table = "COPY q4 FROM '/home/ubuntu/data/Q4_texas_2011.csv' DELIMITER E'\t' CSV"

To create these tables and load them, use the create_table_query strings for each quarter to creat the tables with the appropriate fields. Then, use the load_table strings to load the CSVs into the tables.

In [2]:
# Postgres info to connect
connection_args = {
    'host': input(), # my public ip address
    'user': 'ubuntu',    # username
    'dbname': 'texas_discharge',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

connection = pg.connect(**connection_args)

In [3]:
query = "SELECT * FROM q1 LIMIT 5;"

q1_sample = pd_sql.read_sql(query, connection)

In [4]:
diag_columns = ['admitting_diagnosis']

# get all the diagnosis code fields, exclude poa fields (which stands for present on arrival)
diag_columns.extend([col for col in q1_sample.columns if 'diag_code' in col and 'poa' not in col])

In [5]:
diag_columns_cardiac_arrest = [col + "='4275'" for col in diag_columns]

diag_cardiac_arrest_string = " OR ".join(diag_columns_cardiac_arrest)

In [6]:
# Let's pull the subset of patients who have the diagnosis code for 'cardiac arrest' in any of their diag_column
# fields. The diagnosis code for cardiac arrest is 427.5, which will just be 4275 in the database.

cardiac_arrest_query = "SELECT * FROM {} WHERE " + diag_cardiac_arrest_string

In [7]:
q1_cardiac_arrest_df = pd_sql.read_sql(cardiac_arrest_query.format('q1'), connection)

q2_cardiac_arrest_df = pd_sql.read_sql(cardiac_arrest_query.format('q2'), connection)

q3_cardiac_arrest_df = pd_sql.read_sql(cardiac_arrest_query.format('q3'), connection)

q4_cardiac_arrest_df = pd_sql.read_sql(cardiac_arrest_query.format('q4'), connection)

In [8]:
q1_cardiac_arrest_df.shape

(3579, 194)

In [9]:
q2_cardiac_arrest_df.shape

(3149, 194)

In [10]:
q3_cardiac_arrest_df.shape

(3075, 194)

In [11]:
q4_cardiac_arrest_df.shape

(3380, 194)

In [12]:
with open("pkls/cardiac_arrest_dfs.pkl", "wb") as picklefile:
    pickle.dump([q1_cardiac_arrest_df, q2_cardiac_arrest_df, q3_cardiac_arrest_df, q4_cardiac_arrest_df], picklefile)

Now, let's try pulling the data for all the patients who had influenza. It actually might be more interesting to look at patients who were either admitted or had a principal diagnosis of influenza, rather than people who have influenza in any of their diagnoses.

In [33]:
adm_princ_diag = ['admitting_diagnosis', 'princ_diag_code']

diag_columns_influenza = [col + " IN ('4871','4870','4878')" for col in adm_princ_diag]

diag_influenza_string = " OR ".join(diag_columns_influenza)

In [34]:
diag_influenza_string

"admitting_diagnosis IN ('4871','4870','4878') OR princ_diag_code IN ('4871','4870','4878')"

In [35]:
influenza_query = "SELECT * FROM {} WHERE " + diag_influenza_string

In [38]:
quarters = ['q1', 'q2', 'q3', 'q4']

influenza_dfs = {}

for q in quarters:
    influenza_dfs[q] = pd_sql.read_sql(influenza_query.format(q), connection)

In [46]:
master_influenza_df = pd.concat([influenza_dfs['q1'], influenza_dfs['q2'], influenza_dfs['q3'], influenza_dfs['q4']], ignore_index=True)

In [47]:
with open("pkls/master_influenza_df.pkl", "wb") as picklefile:
    pickle.dump(master_influenza_df, picklefile)

In [54]:
target = 'pat_status'

In [55]:
pat_status_dict = {
    "01": "Discharged to home or self-care (routine discharge)",
    "02": "Discharged to other short term general hospital", 
    "03": "Discharged to skilled nursing facility",
    "04": "Discharged to intermediate care facility",
    "05": "Discharged/transferred to a Designated Cancer Center or Children's Hospital (effective 10-1-2007)",
    "06": "Discharged to care of home health service",
    "07": "Left against medical advice",
    "08": "Discharged to care of Home IV provider",
    "09": "Admitted as inpatient to this hospital",
    "20": "Expired",
    "30": "Still patient",
    "40": "Expired at home",
    "41": "Expired in a medical facility",
    "42": "Expired, place unknown",
    "43": "Discharged/transferred to federal health care facility",
    "50": "Discharged to hospice–home",
    "51": "Discharged to hospice–medical facility",
    "61": "Discharged/transferred within this institution to Medicare-approved swing bed",
    "62": "Discharged/transferred to inpatient rehabilitation facility",
    "63": "Discharged/transferred to Medicare-certified long term care hospital",
    "64": "Discharged/transferred to Medicaid-certified nursing facility",
    "65": "Discharged/transferred to psychiatric hospital or psychiatric distinct part of a hospital",
    "66": "Discharged/transferred to Critical Access Hospital (CAH)",
    "71": "Discharged/transferred to other outpatient service",
    "72": "Discharged/transferred to institution outpatient",
    "`": "Invalid"
}

In [63]:
target_value_counts = master_influenza_df[target].value_counts()

new_index = []

for i in target_value_counts.index:
    new_index.append(pat_status_dict[i])
    
target_value_counts.index = new_index

In [64]:
target_value_counts

Discharged to home or self-care (routine discharge)                                                  2654
Discharged to skilled nursing facility                                                                185
Discharged to care of home health service                                                             166
Discharged to intermediate care facility                                                               39
Expired                                                                                                35
Discharged/transferred to Medicare-certified long term care hospital                                   25
Discharged/transferred to inpatient rehabilitation facility                                            24
Discharged to other short term general hospital                                                        23
Left against medical advice                                                                            14
Discharged/transferred within this institution

In [58]:
master_influenza_df.shape

(3228, 194)